# 06. Follow-Up Questions

Allow follow-up questions to be answered.

In [1]:
import os
import openai
import pinecone

from tqdm.auto import tqdm
from dotenv import load_dotenv
from IPython.core.display import Markdown

/home/vscode/.local/lib/python3.8/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Configure Environment

In [2]:
load_dotenv()

True

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_CHAT_MODEL = os.getenv('OPENAI_CHAT_MODEL', 'gpt-3.5-turbo')
OPENAI_EMBED_MODEL = os.getenv('OPENAI_EMBED_MODEL', 'text-embedding-ada-002')

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_INDEX = os.getenv('PINECONE_INDEX', 'openai-dsm100-2022-oct-transcriptions')
PINECONE_ENV = os.getenv('PINECONE_ENV', 'us-east1-gcp')

In [4]:
# configure openai
openai.api_key = OPENAI_API_KEY

# configure pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

## Functions

In [5]:
def get_pinecone_index(index_name: str) -> pinecone.index.Index:
    # connect to index
    index = pinecone.Index(index_name)
    
    # view index stats
    print(index.describe_index_stats())  

    return index  

# test the function
index = get_pinecone_index(PINECONE_INDEX)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2972}},
 'total_vector_count': 2972}


In [6]:
def get_text_embeddings(text: str) -> list:
    result = openai.Embedding.create(
            input=[text], engine=OPENAI_EMBED_MODEL)

    return result['data'][0]['embedding']

In [7]:
def query_pinecone_index(index: pinecone.index.Index,
                         query: str,
                         k: int = 5,
                         include_metadata=True) -> list:
    # get embeddings for query
    query_embedding = get_text_embeddings(query)

    # query index
    return index.query(
        vector=query_embedding,
        top_k=k,
        include_metadata=include_metadata
    )

In [8]:
def create_prompt(index: pinecone.index.Index,
                  query: str, 
                  prompt_limit: int = 3750) -> str:
    prompt = ""
    
    # query pinecone for matching transcripts
    pinecone_results = query_pinecone_index(index, query)

    # get relevant contexts
    contexts = [
        x['metadata']['text'] for x in pinecone_results['matches']
    ]

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )

    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= prompt_limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )

    return prompt, pinecone_results



In [9]:
def get_chat_response(message_history: list) -> str:
    response = openai.ChatCompletion.create(
        model=OPENAI_CHAT_MODEL,
        messages=message_history
    )

    return response['choices'][0]['message']['content']

In [10]:
def ask_question(index: pinecone.index.Index,
                  query: str,
                  message_history: list = [],
                  prompt_limit: int = 3750):

    # create prompt
    prompt, context = create_prompt(index, query, prompt_limit)

    # add the question to the messages to send
    messages = message_history.copy()
    messages.append({ 'role': 'user', 'content': prompt })

    # return the answer and context
    return get_chat_response(messages), context

# test the function
#answer, _ = ask_question(index, questions['A_1_a'])
#answer

In [11]:
def get_standard_answer(question:str,
                        prompt:str='Please answer the following exam question:',
                        message_history: list = []) -> str:
    """
    Answer the question using only the standard GPT model.
    """
    query = prompt + '\n\n' + question
    response = get_chat_response(
        message_history +
        [{ 'role': 'user', 'content': query }]
    )

    return response

# test the function
#get_standard_answer(questions['A_1_a'])

In [12]:
def get_textbook_answer(question:str,
                        message_history: list = []) -> str:   
    return get_chat_response(
        [{ 'role': 'system', 'content': 'You are a student answering questions using only the textbook \"Artificial Intelligence: A Modern Approach. (Harlow: Pearson, 2016) 3rd edition\" as your background knowledge.'}] +
        message_history +
        [{ 'role': 'user', 'content': question }]
    )

#get_textbook_answer(question)    

## Question Definitions

In [13]:
def display_answer(index: pinecone.index.Index,
                   question: str,
                   message_history: list = []) -> dict:
   answers = {}
   output = []

   output.append('### Question')
   output.append(question)

   output.append('### Answer')
   answer, context = ask_question(index, question, message_history=message_history)
   answers['answer'] = answer
   output.append(answer)

   output.append('### Generic Answer')
   standard_answer = get_standard_answer(question, message_history=message_history)
   answers['generic'] = standard_answer
   output.append(standard_answer)

   output.append('### Textbook Answer')
   textbook_answer = get_textbook_answer(question, message_history=message_history)
   answers['textbook'] = textbook_answer
   output.append(textbook_answer)

   output.append('### Context')
   context_list = set(x['metadata']['topic'] + '::' + x['metadata']['title'] for x in context['matches'])
   for item in context_list:
      output.append('- ' + item)

   display(Markdown('\n'.join(output)))

   return answers

# test the function
#answers = display_answer(index, question)


In [14]:
questions = {}

In [15]:
question = """
AI has a long history with ups and downs, over-optimistic expectations, and the
industrial period. At present, AI is in the era of:
[3 marks]

Select ALL statements that apply.
i. Big data.
ii. Deep learning.
iii. Robotics.
iv. General AI.
"""

questions['A_1_a'] = question
answers = display_answer(index, question)

### Question

AI has a long history with ups and downs, over-optimistic expectations, and the
industrial period. At present, AI is in the era of:
[3 marks]

Select ALL statements that apply.
i. Big data.
ii. Deep learning.
iii. Robotics.
iv. General AI.

### Answer
i. Big data.
ii. Deep learning.
### Generic Answer


i. Big data.
ii. Deep learning.

Both statements i and ii apply to the current era of AI.
### Textbook Answer
i. Big data.
ii. Deep learning.
### Context
- Topic 7: Robot Scientist::Lecture 1: Discovery science and the Robot Scientist system
- Topic 1: Introduction::Lecture 2:  Milestones in the history of AI
- Topic 1: Introduction::Lecture 7: State-of-the-art in AI
- Module information::Module introduction video

In [16]:
question = """
b. It is important for AI systems to be explainable and interpretable because it:
Select ALL statements that apply.
[3 marks]

i. Gives users confidence in the system.
ii. Is a legal requirement.
iii. Helps in safeguarding against bias.
iv. Enables assessing of system’s robustness and vulnerability. 
"""

questions['A_1_b'] = question
answers = display_answer(index, question)

### Question

b. It is important for AI systems to be explainable and interpretable because it:
Select ALL statements that apply.
[3 marks]

i. Gives users confidence in the system.
ii. Is a legal requirement.
iii. Helps in safeguarding against bias.
iv. Enables assessing of system’s robustness and vulnerability. 

### Answer
i. Gives users confidence in the system.
iii. Helps in safeguarding against bias.
iv. Enables assessing of system’s robustness and vulnerability.
### Generic Answer


i. Gives users confidence in the system.
iii. Helps in safeguarding against bias.
iv. Enables assessing of system’s robustness and vulnerability.
### Textbook Answer
i. Gives users confidence in the system.
iii. Helps in safeguarding against bias.
iv. Enables assessing of system’s robustness and vulnerability.
### Context
- Topic 1: Introduction::Lecture 10: Explainable AI
- Topic 1: Introduction::Lecture 8: Ethical and legal issues

In [17]:
question = """
c. Heuristic functions can be generated based on:
Select ALL statements that apply.
[3 marks]

i. Simplified (or relaxed) problems.
ii. Forecasted blind search results.
iii. Previous experience.
iv. Features of the search space.
"""

questions['A_1_c'] = question
answers = display_answer(index, question)

### Question

c. Heuristic functions can be generated based on:
Select ALL statements that apply.
[3 marks]

i. Simplified (or relaxed) problems.
ii. Forecasted blind search results.
iii. Previous experience.
iv. Features of the search space.

### Answer
i. Simplified (or relaxed) problems.
iii. Previous experience.
iv. Features of the search space.
### Generic Answer


i. Simplified (or relaxed) problems.
iii. Previous experience.
iv. Features of the search space.
### Textbook Answer


i. Simplified (or relaxed) problems.
iii. Previous experience.
iv. Features of the search space.

Heuristic functions can be generated based on simplified versions of the problem, previous experience with similar problems, and features of the search space that suggest the desirability of one branch over another. Forecasted blind search results are not as relevant to the generation of heuristic functions.
### Context
- Topic 2: Problem solving agents::Lecture 3: Informed search

In [18]:
question = """
d. Induction is deductive reasoning based on generalisation of facts.
[3 marks]

Is this statement is True/False?
"""

questions['A_1_d'] = question
answers = display_answer(index, question)

### Question

d. Induction is deductive reasoning based on generalisation of facts.
[3 marks]

Is this statement is True/False?

### Answer
False. Induction is a type of reasoning that involves generalization of facts, but it is not deductive reasoning.
### Generic Answer


False. 

Induction is a form of reasoning that moves from specific observations to broader generalizations and is often used to develop scientific theories or hypotheses. It is not deductive reasoning, which moves from general principles or premises to specific conclusions.
### Textbook Answer
This statement is False. 

Induction is a form of logical reasoning that infers a general rule or principle from a set of specific observations. It involves moving from specific instances to a general conclusion, whereas deduction is the process of deriving a specific conclusion from a general principle or rule.
### Context
- Topic 3: Knowledge, reasoning and planning::Lecture 6: Logical reasoning
- Topic 8: Robot Scientist part 2::Lecture 2: Hypotheses generation: induction

In [19]:
question = """
e. A genetic algorithm:
Select ALL statements that apply.
[3 marks]

i. Is a global search algorithm
ii. Is a local search algorithm
iii. Is inspired by Charles Darwin’s theory of natural evolution.
iv. Includes three evolutionary processes: selection, gene crossover, and
mutation.
"""

questions['A_1_e'] = question
answers = display_answer(index, question)

### Question

e. A genetic algorithm:
Select ALL statements that apply.
[3 marks]

i. Is a global search algorithm
ii. Is a local search algorithm
iii. Is inspired by Charles Darwin’s theory of natural evolution.
iv. Includes three evolutionary processes: selection, gene crossover, and
mutation.

### Answer
iii. Is inspired by Charles Darwin’s theory of natural evolution.
iv. Includes three evolutionary processes: selection, gene crossover, and
mutation.
### Generic Answer


i. Is a global search algorithm
iii. Is inspired by Charles Darwin’s theory of natural evolution.
iv. Includes three evolutionary processes: selection, gene crossover, and mutation.

(These statements are all true about genetic algorithms.)
### Textbook Answer


i. Is a global search algorithm: True
iii. Is inspired by Charles Darwin’s theory of natural evolution: True
iv. Includes three evolutionary processes: selection, gene crossover, and
mutation: True

Therefore, the correct options are i, iii, and iv.
### Context
- Topic 2: Problem solving agents::Lecture 4: Genetic algorithms

In [20]:
question = """
f. The more background knowledge an intelligent agent can use – the more intelligent
behaviour it can exhibit. Background knowledge can include:
Select ALL statements that apply.
[3 marks]

(i). Problem-specific knowledge
(ii). Declarative knowledge
(iii). Procedural knowledge
(iv). A dataset
"""

questions['A_1_f'] = question
answers = display_answer(index, question)

### Question

f. The more background knowledge an intelligent agent can use – the more intelligent
behaviour it can exhibit. Background knowledge can include:
Select ALL statements that apply.
[3 marks]

(i). Problem-specific knowledge
(ii). Declarative knowledge
(iii). Procedural knowledge
(iv). A dataset

### Answer
(i). Problem-specific knowledge
(ii). Declarative knowledge 
(iii). Procedural knowledge
### Generic Answer


(i). Problem-specific knowledge 
(ii). Declarative knowledge 
(iii). Procedural knowledge
### Textbook Answer
(i). Problem-specific knowledge

(ii). Declarative knowledge

(iii). Procedural knowledge
### Context
- Topic 3: Knowledge, reasoning and planning::Lecture 1: Knowledge-based agents
- Topic 2: Problem solving agents::Lecture: Topic 2 summary

In [21]:
question = """
g. An intelligent agent needs to handle uncertainty only when the environment is both:
partially observable and nondeterministic
[3 marks]

Is this statement is True/False?
"""

questions['A_1_g'] = question
answers = display_answer(index, question)

### Question

g. An intelligent agent needs to handle uncertainty only when the environment is both:
partially observable and nondeterministic
[3 marks]

Is this statement is True/False?

### Answer
True
### Generic Answer


True
### Textbook Answer
The statement is true. According to the textbook, an intelligent agent needs to handle uncertainty only when the environment is both partially observable and nondeterministic. In partially observable environments, the agent does not have access to the complete state of the environment, and in nondeterministic environments, the next state of the environment cannot be predicted with certainty given the current state and action. Therefore, uncertain reasoning is required to handle such environments.
### Context
- Topic 2: Problem solving agents::Lecture 5: Nondeterministic search, partially observable space
- Topic 4: Quantifying uncertainty::Lecture 1: Quantifying uncertainty

In [22]:
question = """
The principle of maximum expected utility (MEU) says that a rational agent should
choose:
Select ONE most appropriate answer.
[3 marks]

(i). The action that maximises the agent’s current utility.
(ii). The action that maximises the agent’s expected utility.
(iii). The utility function that maximises the expected agent’s value.
(iv). The utility function that maximises the current agent’s value.
"""

questions['A_1_h'] = question
answers = display_answer(index, question)

### Question

The principle of maximum expected utility (MEU) says that a rational agent should
choose:
Select ONE most appropriate answer.
[3 marks]

(i). The action that maximises the agent’s current utility.
(ii). The action that maximises the agent’s expected utility.
(iii). The utility function that maximises the expected agent’s value.
(iv). The utility function that maximises the current agent’s value.

### Answer
(ii). The action that maximises the agent's expected utility.
### Generic Answer


(ii). The action that maximises the agent’s expected utility.
### Textbook Answer


The principle of maximum expected utility (MEU) states that a rational agent should choose the action that maximizes the agent's expected utility. Therefore, the answer is (ii) - The action that maximises the agent's expected utility.
### Context
- Topic 4: Quantifying uncertainty::Lecture 5: Decision making
- Topic 4: Quantifying uncertainty::Lecture 1: Quantifying uncertainty

In [23]:
question = """
i. The Bayesian approach to the source and status of probability numbers is:
Select ONE most appropriate answer.
[3 marks]

(i). Subjective.
(ii). Objective.
(iii). Frequentist.
(iv). Statistical.
"""

questions['A_1_i'] = question
answers = display_answer(index, question)

### Question

i. The Bayesian approach to the source and status of probability numbers is:
Select ONE most appropriate answer.
[3 marks]

(i). Subjective.
(ii). Objective.
(iii). Frequentist.
(iv). Statistical.

### Answer
(i). Subjective.
### Generic Answer


(i). Subjective.
### Textbook Answer
(i) Subjective.
### Context
- Topic 4: Quantifying uncertainty::Lecture: Introduction to Topic 4
- Topic 4: Quantifying uncertainty::Lecture 3: Bayes rule
- Topic 4: Quantifying uncertainty::Lecture 1: Quantifying uncertainty

In [24]:
question = """
j. A typical cycle of scientific discovery includes the following steps:
Select ALL statements that apply.
[3 marks]

i. Conducting surveys.
ii. Hypothesis generation.
iii. Deduction – a prediction from the hypothesis about future effects or
experimental results
iv. Hypothesis testing, to verify or disprove predictions.
"""

questions['A_1_j'] = question
answers = display_answer(index, question)

### Question

j. A typical cycle of scientific discovery includes the following steps:
Select ALL statements that apply.
[3 marks]

i. Conducting surveys.
ii. Hypothesis generation.
iii. Deduction – a prediction from the hypothesis about future effects or
experimental results
iv. Hypothesis testing, to verify or disprove predictions.

### Answer
ii. Hypothesis generation.
iv. Hypothesis testing, to verify or disprove predictions.
### Generic Answer


ii. Hypothesis generation.
iii. Deduction – a prediction from the hypothesis about future effects or experimental results.
iv. Hypothesis testing, to verify or disprove predictions.
### Textbook Answer
ii. Hypothesis generation.
iv. Hypothesis testing, to verify or disprove predictions. 

These are the two steps mentioned in the textbook that apply to a typical cycle of scientific discovery. There is no mention of conducting surveys or deduction as separate steps in this process.
### Context
- Topic 3: Knowledge, reasoning and planning::Lecture 6: Logical reasoning
- Topic 7: Robot Scientist::Lecture 2: The concept of Robot Scientist
- Topic 8: Robot Scientist part 2::Lecture 1: Hypotheses generation: abduction
- Topic 8: Robot Scientist part 2::Lecture 2: Hypotheses generation: induction

In [25]:
question = """
a) What type of problems are Markov Decision Processes (MDPs) suitable for
modelling?
"""

questions['B_2_a'] = question
answers = display_answer(index, question)

### Question

a) What type of problems are Markov Decision Processes (MDPs) suitable for
modelling?

### Answer
MDPs are suitable for modelling stochastic sequential decision problems.
### Generic Answer


MDPs are suitable for modelling problems involving decision-making in uncertain environments, where the outcome of an action is probabilistic and dependent on the current state. Examples of such problems include inventory control, robot navigation, financial portfolio optimization, game AI, etc. MDPs are particularly useful in modeling problems where optimal decisions can be made by considering long-term trade-offs between immediate rewards and future outcomes.
### Textbook Answer
Markov Decision Processes (MDPs) are suitable for modeling decision-making problems in situations where the outcome of an action is uncertain and partially dependent on the current state of the system. Specifically, MDPs are used to model problems where the decision-making agent aims to maximize a cumulative reward over a series of decision points while interacting with an uncertain environment over time. Examples of problems that can be modeled using MDPs include robotics, game playing, control of dynamic systems, and decision making under uncertainty in general.
### Context
- Topics 9&10: Game Player::Lecture: Formalising the problem

In [26]:
question = """
b) Choose a video game that is significantly different from the Breakout game
seen in the module.
[2 marks]

• Name and briefly describe the game you have chosen.
• What are the control inputs?
• How can you gain points in the game?
• How might an artificial intelligence system perceive the state of the
game? Please do not use the example of screen pixels - think about
the underlying state of the game. 
"""

questions['B_2_b'] = question
answers = display_answer(index, question)

# set the message history
message_history = []
message_history.append({ 'role': 'user', 'content': question })
message_history.append({ 'role': 'assistant', 'content': answers['textbook'] })


### Question

b) Choose a video game that is significantly different from the Breakout game
seen in the module.
[2 marks]

• Name and briefly describe the game you have chosen.
• What are the control inputs?
• How can you gain points in the game?
• How might an artificial intelligence system perceive the state of the
game? Please do not use the example of screen pixels - think about
the underlying state of the game. 

### Answer


As an AI language model, I cannot see or hear videos, however, I can provide an example of a video game that is significantly different from Breakout. 

Game: Super Mario Bros.
Description: Super Mario Bros. is a classic side-scrolling platformer game where players control Mario, the plumber, in his quest to rescue Princess Toadstool from the evil Bowser. 
Control Inputs: Players use the directional pad to move Mario left, right, and jump to avoid obstacles and enemies. 
Gaining Points: Players can gain points by collecting coins, power-ups, and defeating enemies. 
AI Perception: An AI system could perceive the state of the game through variables such as Mario's position, the location of enemies and obstacles, the number of coins collected and power-ups obtained, and the time remaining in the level.
### Generic Answer


I'm sorry, as an AI language model, I do not have the ability to see the Breakout game from the module. However, I can provide an answer based on a different game.

• Name and briefly describe the game you have chosen.
Game: Terraria
Terraria is a 2D side-scrolling action-adventure sandbox game, where players explore, craft, build, and fight to progress through the game.

• What are the control inputs?
The control inputs include arrow keys to move, spacebar to jump, mouse clicks to attack, and various keyboard inputs to access inventory, crafting, and other menus.

• How can you gain points in the game?
Points are not given in Terraria, however, players can gain items, weapons, and accessories as they progress through the game. They can also accumulate coins by finding them in chests, selling items to NPCs, or defeating enemies.

• How might an artificial intelligence system perceive the state of the game? Please do not use the example of screen pixels - think about the underlying state of the game.
An artificial intelligence system would perceive the state of the game by considering the player's inventory, the player's current location in the game world, what items the player has collected, how many enemies the player has defeated, and the player's health and mana levels. The AI could also analyze the terrain and weather conditions to give the player better advice on how to navigate the game world.
### Textbook Answer


I have chosen the game "The Legend of Zelda: Breath of the Wild", which is an action-adventure game developed by Nintendo. In this game, the player controls a character named Link, who explores an open world and completes various quests.

The control inputs involve various buttons on a game controller. The player can move Link, make him jump, attack enemies, use tools such as a bow and arrow or bombs, and interact with the environment.

The player can gain points, or rather, progress in the game by completing quests and defeating enemies. Moreover, the game also includes a system of rewards for exploring the in-game world and discovering new locations.

An artificial intelligence system might perceive the state of the game as a set of variables, such as Link's health, inventory of tools and weapons, available quests, and the position of enemies and objects in the environment. It might also analyze the relationships between these variables and use them to decide the best course of action for the player, such as which quest to pursue or which enemy to attack first.
### Context
- Topics 9&10: Game Player::Lecture: How might we build a game playing AI?
- Topics 9&10: Game Player::Lecture: Formalising the problem
- Topics 9&10: Game Player::Lecture: Game player case study introduction

In [27]:
question = """
c) Name and describe THREE elements of a Markov Decision Process. For
each element, say what it represents from your chosen video game.
[3 marks]
"""

questions['B_2_c'] = question
answers = display_answer(index, question, message_history=message_history)

# update the message history
message_history.append({ 'role': 'user', 'content': question })
message_history.append({ 'role': 'assistant', 'content': answers['textbook'] })

### Question

c) Name and describe THREE elements of a Markov Decision Process. For
each element, say what it represents from your chosen video game.
[3 marks]

### Answer
Three elements of a Markov Decision Process are:

1. States: These represent the different situations or circumstances that the agent can find themselves in the game. In "The Legend of Zelda: Breath of the Wild", the states could represent Link's location, his current health, his inventory of tools and weapons, and whether he is in the middle of completing a quest or not.

2. Actions: These are the different choices or decisions that the agent can make in each state. In the game, these could represent whether Link chooses to move forward, backward or sideways, whether he wants to attack enemies or use tools, such as a bow and arrow or bombs.

3. Rewards: These are the positive or negative feedback that the agent receives for taking a particular action. In the game, rewards could represent gaining health or obtaining valuable tools by completing a quest or losing health by getting attacked by enemies.
### Generic Answer


The three elements of a Markov Decision Process are:

1) States: These represent the different possible situations that the game can be in. In the game "The Legend of Zelda: Breath of the Wild", states could include things like Link's health, his inventory of weapons and tools, his location in the game world, and the positions of enemies and obstacles in his immediate vicinity.

2) Actions: These represent the possible moves that the player can make in each state. In the game, actions might include movement in different directions, use of different weapons or tools, or interaction with objects in the game world.

3) Rewards: These represent the outcomes of each action in each state, and they determine the player's ultimate score or success in the game. In "The Legend of Zelda: Breath of the Wild", rewards might include experience points for defeating enemies, items or upgrades for completing quests or exploring new areas, or penalties for taking damage or failing a challenge.

Overall, a Markov Decision Process can be used to model the player's decision-making process as they navigate through the game world, taking into account the different states, actions, and rewards that they encounter along the way. It can also be used to optimize the player's strategy by finding the best sequence of actions to take in each situation to maximize their score or success in the game.
### Textbook Answer
1. States: In a Markov Decision Process, states refer to the different configurations of the environment that the agent can be in. In "The Legend of Zelda: Breath of the Wild", the states could represent the different locations on the map, the health status of the player's character, the types of enemies nearby, and the weather conditions.

2. Actions: Actions are the possible decisions that an agent can make at any given state. In the game, the actions could correspond to the different ways that the player can choose to interact with the environment, including movement, attacking enemies, using tools such as a bow and arrow or bombs, and interacting with NPCs.

3. Rewards: Rewards represent the consequences of an agent's actions in a given state. In the game, rewards could represent the points gained and the quest completion status. Additionally, rewards could also represent the benefits and costs of different actions, such as gaining health by eating food or losing health by falling from a great height. These rewards inform the agent's decision-making process in subsequent states.
### Context
- Topics 9&10: Game Player::Lecture: Formalising the problem

In [28]:
question = """
d) Do you think a Markov Decision Process is a suitable way to model playing
your chosen game? Justify your answer.
[2 marks]
"""

questions['B_2_d'] = question
answers = display_answer(index, question, message_history=message_history)

# set the message history
message_history = []
message_history.append({ 'role': 'user', 'content': question })
message_history.append({ 'role': 'assistant', 'content': answers['textbook'] })

### Question

d) Do you think a Markov Decision Process is a suitable way to model playing
your chosen game? Justify your answer.
[2 marks]

### Answer
Yes, a Markov Decision Process could be a suitable way to model playing "The Legend of Zelda: Breath of the Wild". The game involves making a series of decisions, with actions that affect the state of the environment and in turn, the future decisions that can be made. Additionally, the game has probabilistic elements, such as the weather and the movements of enemies, that can impact the decision-making process. Furthermore, the game has a reward system that provides incentive to optimize decision-making for long-term success. Therefore, a Markov Decision Process can be a suitable way to formalize and optimize the decision-making process in the game.
### Generic Answer
Yes, I think a Markov Decision Process is a suitable way to model playing "The Legend of Zelda: Breath of the Wild". The game's open-world structure makes it suitable for a Markov Decision Process, as the player's actions and the environment's reactions establish a dynamic relationship between the states, actions, and rewards. Furthermore, the game involves a large set of possible states, actions, and rewards, which could be effectively handled through the efficiency of a Markov Decision Process. Additionally, the player's decisions are interdependent and rely on the current state of the game, as they affect the environment's future state, warranting the use of a Markov Decision Process.
### Textbook Answer
Yes, I think a Markov Decision Process is a suitable way to model playing "The Legend of Zelda: Breath of the Wild". The game involves multiple states, actions, and rewards, and the decisions made by the player at each state affect the future states and rewards. Using a Markov Decision Process can help an artificial intelligence system to make optimal decisions based on the current state and maximize the cumulative rewards over time. The system can learn from experience and adjust its actions to improve its chances of achieving the overall goal of completing the game.
### Context
- Topics 9&10: Game Player::Lecture: Formalising the problem

In [29]:
question = """
e) Reinforcement learning might be described as learning a Q function. If we
are using Q-learning to learn how to play video games, state TWO aspects
that make Q-learning difficult.
[2 marks]
"""

questions['B_2_e'] = question
answers = display_answer(index, question, message_history=message_history)

### Question

e) Reinforcement learning might be described as learning a Q function. If we
are using Q-learning to learn how to play video games, state TWO aspects
that make Q-learning difficult.
[2 marks]

### Answer
Two aspects that make Q-learning difficult when learning how to play video games are:

1. The exploration-exploitation trade-off: In order to learn the optimal Q values, the agent needs to try out different actions in different states. However, it also needs to exploit the Q function to use its knowledge of the optimal actions to maximize its rewards. The challenge is to find a balance between exploration and exploitation.

2. The curse of dimensionality: Video games often require the agent to process large amounts of data from the game environment, such as the pixels on the screen. This leads to a high-dimensional state space, which can make it difficult and inefficient to learn the Q function. The agent needs to extract relevant features from the environment to reduce the dimensionality of the state space.
### Generic Answer
Q-learning can be a challenging task when applied to learning how to play video games due to the following two aspects:
 
1. Exploration-exploitation trade-off - In Q-learning, the algorithm needs to balance exploration and exploitation while choosing actions from the current state. Exploration is required to discover new states that may lead to higher rewards, while exploitation chooses the action that is most likely to result in the highest reward. Finding the right balance between exploration and exploitation is a challenge, as too much of one can lead to missed opportunities, while too much of the other can lead to getting stuck in a suboptimal state.

2. State and action space - Many video games have vast state and action spaces, which can make it difficult to learn the Q function accurately. The algorithm needs to iterate through all possible combinations of states and actions, which can lead to an exponential growth in the number of values to be learned. The size of the state space and action space can also increase with game complexity, adding to the challenge. Additionally, some actions may only result in a reward after several steps, making it hard to associate the reward with the corresponding action.
### Textbook Answer
Two aspects that make Q-learning difficult in learning how to play video games are:

1. Exploration vs. Exploitation Dilemma: In order to learn the optimal Q-function, the Q-learning algorithm often needs to explore and try different actions in different states. However, there is a tradeoff between exploration and exploitation, which means the algorithm needs to balance trying new actions with the more direct path to reaching the goal. This can be challenging to optimize, especially in complex and uncertain environments like video games.

2. High Dimensional State Space: The state space in video games is often high dimensional, meaning there are a large number of possible states. This can make it difficult for the Q-learning algorithm to properly approximate the Q-function, as it needs to be evaluated at each possible state-action pair. This can lead to issues such as overfitting or underfitting the Q-function, which can impact the overall performance of the algorithm.
### Context
- Topics 9&10: Game Player::Lecture: How might we build a game playing AI?
- Topics 9&10: Game Player::Lecture: Value functions and DQN
- Topics 9&10: Game Player::Lecture: Weekly outro
- Topics 9&10: Game Player::Lecture: The loss function for DQN

In [30]:
question = """
Nowadays AI systems are capable of making autonomous scientific discoveries. In
this module, we considered several AI systems capable of making automated
scientific discoveries, for example, the mobile robotic chemist, the automatic
statistician, the robot scientists Adam and Eve. The research in automated scientific
discovery is flourishing, and the target is to develop systems capable of making
significant scientific discoveries.

a) What are the advantages of automated discovery systems?
[3 marks]
"""

questions['B_3_a'] = question
answers = display_answer(index, question)

# set the message history
message_history = []
message_history.append({ 'role': 'user', 'content': question })
message_history.append({ 'role': 'assistant', 'content': answers['textbook'] })

### Question

Nowadays AI systems are capable of making autonomous scientific discoveries. In
this module, we considered several AI systems capable of making automated
scientific discoveries, for example, the mobile robotic chemist, the automatic
statistician, the robot scientists Adam and Eve. The research in automated scientific
discovery is flourishing, and the target is to develop systems capable of making
significant scientific discoveries.

a) What are the advantages of automated discovery systems?
[3 marks]

### Answer
The passage does not specifically mention the advantages of automated discovery systems. Therefore, no points can be awarded for this question.
### Generic Answer


Automated discovery systems offer several advantages over traditional scientific discovery methods. Firstly, they can process and analyze vast amounts of data much faster than humans, leading to more efficient scientific discovery. Secondly, they can identify and explore patterns in data that may not be immediately apparent to human researchers, potentially leading to the discovery of new phenomena or connections. Finally, automated discovery systems can operate continuously without the need for breaks, allowing for round-the-clock scientific discovery.
### Textbook Answer


The advantages of automated discovery systems are:

1. Speed: AI systems can process large volumes of data in a short time, enabling them to discover patterns, trends, and relationships that would take humans much longer to identify.

2. Precision: AI systems can detect subtle changes and anomalies that may be overlooked by human researchers, leading to more accurate and reliable results.

3. Scalability: Automated discovery systems can scale up their operations to work with large datasets or conduct multiple experiments simultaneously, making them ideal for addressing complex research questions.

Overall, automated discovery systems have the potential to accelerate the pace of scientific discovery and advance our understanding of complex phenomena.
### Context
- Topic 7: Robot Scientist::Lecture 2: The concept of Robot Scientist
- Topic 7: Robot Scientist::Lecture 3: History of Robot Scientists and future trends

In [31]:
question = """
b) What are the current limitations of automated discovery systems? 
[3 marks]
"""

questions['B_3_b'] = question
answers = display_answer(index, question, message_history=message_history)

# update the message history
message_history.append({ 'role': 'user', 'content': question })
message_history.append({ 'role': 'assistant', 'content': answers['textbook'] })

### Question

b) What are the current limitations of automated discovery systems? 
[3 marks]

### Answer
The context does not provide specific information about the current limitations of automated discovery systems. Therefore, an answer cannot be provided based on the given context.
### Generic Answer


The current limitations of automated discovery systems include:

1. Interpretability: Some AI models are complex and lack transparency, making it difficult to understand how they arrived at their findings. This could hinder researchers' ability to validate the results or build on the discoveries.

2. Bias: Automated discovery systems are only as good as the data they are trained on. If the input data is biased or incomplete, the AI model could produce flawed results. This could lead to reinforcing existing biases or overlooking important factors that could influence the research outcomes.

3. Creativity: Automated discovery systems are good at pattern recognition and optimization, but they currently lack the ability to think creatively or form new ideas. They are limited to exploring the patterns and variables that are programmed into them and may miss unexpected or unpredictable phenomena.

Overall, while automated discovery systems have the potential to transform scientific discovery, they are still limited by the technologies and methodologies available to researchers. Efforts are underway to address these limitations, and ongoing research will continue to improve the scope and scalability of automated discovery systems.
### Textbook Answer
The current limitations of automated discovery systems are:

1. Narrow focus: AI systems are designed to address specific research questions or tasks, and they may not have the flexibility to adapt to new or unexpected problems.

2. Lack of creativity: AI systems rely on predefined algorithms and statistical methods to generate hypotheses and conduct experiments, which may limit their capacity for creative problem-solving.

3. Ethics and bias: AI systems may perpetuate biases or ethical concerns present in the data they are trained on or in the programming itself, affecting the validity and reliability of the results.

Overall, while automated discovery systems have shown great promise, they still have several limitations that need to be addressed to maximize their potential as scientific research tools.
### Context
- Topic 7: Robot Scientist::Topic 7 summary
- Topic 7: Robot Scientist::Lecture 3: History of Robot Scientists and future trends

In [32]:
question = """
c) Describe the functionality of one of the key components of the robot scientist
system, e.g. a hypotheses generator, a reasoning engine, an experiments
planner, or data analysis component. What is required for the functioning of
the selected component? What are its input and output? How does it interact
with other components of the system? Outline the workings of the component.
[4 marks]
"""

questions['B_3_c'] = question
answers = display_answer(index, question, message_history=message_history)

### Question

c) Describe the functionality of one of the key components of the robot scientist
system, e.g. a hypotheses generator, a reasoning engine, an experiments
planner, or data analysis component. What is required for the functioning of
the selected component? What are its input and output? How does it interact
with other components of the system? Outline the workings of the component.
[4 marks]

### Answer
One of the key components of the robot scientist system is the hypotheses generator. This component is responsible for generating potential explanations or hypotheses to explain a given scientific phenomenon. The hypotheses generator uses both deductive and abductive reasoning methods to formulate new hypotheses based on existing knowledge and available data. 

The hypotheses generator component requires access to a wide range of data sources and analytical tools to identify patterns and trends that could lead to new hypotheses. The input to this component includes all available data and information related to the scientific phenomenon being investigated. The output of the hypotheses generator is a list of potential explanations or hypotheses that could explain the phenomenon. 

The hypotheses generator interacts closely with the planning agent, which designs experiments to test the generated hypotheses. The planning agent produces an experimental protocol that is used by the automated laboratory to carry out thousands of experiments. The results of these experiments are then passed to the data analysis and interpretation agent, which decides whether the generated hypothesis is confirmed or not.

Overall, the hypotheses generator is a critical component of the robot scientist system as it facilitates the generation of new hypotheses and can help to accelerate the pace of scientific discovery.
### Generic Answer
One of the key components of the robot scientist system is the hypotheses generator, which is responsible for developing scientific hypotheses based on available data. The hypotheses generator relies on a combination of statistical analysis and machine learning algorithms to identify patterns and relationships in the data that may be indicative of new scientific hypotheses.

To function properly, the hypotheses generator requires a large dataset of scientific data and metadata, as well as a set of predefined rules and parameters that govern the types of hypotheses that can be generated. The input to the hypotheses generator is the dataset of scientific data, while the output is a set of suggested scientific hypotheses.

The hypotheses generator must interact closely with the experiment planner component, which uses the generated hypotheses to develop new experiments that test the validity of the hypotheses. Additionally, the reasoning engine component must be able to evaluate the hypotheses generated by the hypotheses generator to determine their scientific merit and prioritize them for further experimentation.

The workings of the hypotheses generator component involve several steps, including data preprocessing, feature extraction, and machine learning analysis. First, the dataset of scientific data is cleaned and normalized to prepare it for analysis. Next, features are extracted from the data using a variety of statistical and signal processing techniques. The machine learning algorithms then analyze the extracted features to identify patterns and relationships in the data, and generate new scientific hypotheses based on these patterns.

Overall, the hypotheses generator is a crucial component of the robot scientist system that enables the system to generate new scientific knowledge automatically. By analyzing large volumes of data and proposing novel scientific hypotheses, the hypotheses generator has the potential to revolutionize the way we conduct scientific research.
### Textbook Answer
One of the key components of the robot scientist system is the hypotheses generator. The hypotheses generator is responsible for proposing new hypotheses that can be tested through further experimentation. This component uses machine learning algorithms to analyze data, identify patterns, and generate new hypotheses that could explain the observed phenomena.

The inputs to the hypotheses generator are typically experimental data and background knowledge from various sources, such as scientific literature databases or ontologies. The output of the hypotheses generator is a set of candidate hypotheses that are ranked based on their plausibility and potential to explain the data.

The hypotheses generator interacts closely with other components of the robot scientist system, such as the reasoning engine, experiments planner, and data analysis component. The hypotheses generated by the hypotheses generator are evaluated by the reasoning engine, which checks if they are consistent with the established scientific principles and rules. If a hypothesis passes this check, the experiments planner will propose new experiments to test it. The data analysis component is responsible for processing the results of the experiments and updating the hypotheses generated.

The hypotheses generator works by analyzing the data and identifying patterns that may be indicative of an underlying causal relationship. This component uses various machine learning algorithms, such as clustering or association rule mining, to identify these patterns. The hypotheses generator then formulates new hypotheses that can potentially explain the observed patterns. The hypotheses generator further refines these hypotheses by considering prior knowledge and testing them against existing data. This iterative process continues until the best candidate hypotheses are identified and proposed for further testing.
### Context
- Topic 8: Robot Scientist part 2::Lecture: Topic 8 summary
- Topic 7: Robot Scientist::Lecture 5: Overview of the toy system: components and their interactions
- Topic 8: Robot Scientist part 2::Introduction to Topic 8

In [33]:
question = """
a) What is a genetic algorithm?
[1 mark]
"""

questions['B_4_a'] = question
answers = display_answer(index, question)

### Question

a) What is a genetic algorithm?
[1 mark]

### Answer
A genetic algorithm is a probabilistic search procedure designed to work on large spaces involving states that can be represented as strings, and is inspired by Charles Darwin's theory of natural evolution. It includes three evolutionary processes, selection, gene crossover, and mutation. There are five components to a genetic algorithm - the initial population, fitness function, selection, crossover, and mutation. The algorithm increases the population of chromosomes, a set of potential solutions for the problem, and selects the fittest individuals for further reproduction. Offsprings inherit characteristics of their parents and the process continues until convergence.
### Generic Answer


A genetic algorithm is a computational optimization method inspired by natural selection and genetics. It mimics the way biological organisms evolve over time through reproduction, mutation, and selection in order to find the best solution to a particular problem.
### Textbook Answer
A genetic algorithm is a search heuristic inspired by the process of natural selection. It operates on a population of potential solutions applying genetic operators such as selection, crossover, and mutation to produce new generations of solutions. These new solutions are evaluated and the process continues until a satisfactory solution is found.
### Context
- Topics 5&6: Creatures::Lecture: Bioinspired computing
- Topics 5&6: Creatures::Lecture: Introduction to Lesson 1
- Topic 2: Problem solving agents::Lecture 4: Genetic algorithms
- Topics 5&6: Creatures::Lecture: Artificial evolution and genetic algorithms: fitness functions, genetic encoding and manipulation, population models

In [34]:
question = """
b) Draw a diagram which illustrates the runtime of a genetic algorithm. Show the
major components of the software and when they are used.
[5 marks]
"""

questions['B_4_b'] = question
answers = display_answer(index, question)

### Question

b) Draw a diagram which illustrates the runtime of a genetic algorithm. Show the
major components of the software and when they are used.
[5 marks]

### Answer


Sorry, as an AI language model, I am unable to draw diagrams. However, based on the context provided, it seems that the genetic algorithm includes five components: initial population, fitness function, selection, crossover, and mutation. The process involves selecting the best-fitting individuals from the initial population, exchanging their genetic information (crossover), introducing mutations, and repeating the process until an optimal or satisfactory solution is found. The diagram mentioned in the context also shows the importance of variation in genetic algorithms, which plays a key role in generating diversity and exploring different solutions.
### Generic Answer


As an AI language model, I am not able to draw diagrams, but I can describe the major components involved in a genetic algorithm runtime. A genetic algorithm runtime consists of the following major components:

1. Initialization: In this first step, the initial population is created randomly or based on some prior knowledge. The size of the population, the encoding of the solution, and the fitness function are specified.

2. Selection: The fitter individuals are selected from the population in this step using a fitness function. Selection methods include roulette wheel selection, tournament selection, and rank-based selection.

3. Crossover: In this step, some of the selected individuals are crossed to create a new population. Crossover methods include one-point crossover, two-point crossover, and uniform crossover.

4. Mutation: In this step, some of the individuals in the new population are mutated randomly with a predefined probability. Mutation may involve swapping, swapping randomly or in a pattern, or introducing new genetic material from a random individual.

5. Evaluation: The fitness of each individual in the new population is then evaluated based on the fitness function.

6. Termination: The previous steps are repeated several times until a stopping criteria is met, such as a solution to the problem being found or a maximum number of iterations being reached.

The diagram illustrating the runtime of a genetic algorithm would typically show these steps along a timeline, with arrows indicating the flow of information between the steps.
### Textbook Answer
I'm sorry, as an AI language model, I am not able to draw diagrams. However, I can describe the general components and runtime of a genetic algorithm. 

The runtime of a genetic algorithm consists of several components that are repeated iteratively until a satisfactory solution is found. The major components of a genetic algorithm are:

1. Initialization: In this component, the algorithm initializes the population of potential solutions through random generation or heuristics.

2. Fitness Evaluation: Once a population of potential solutions is established, the algorithm evaluates each individual's fitness or quality. This evaluation provides a numerical score to rank and sort the population.

3. Parent Selection: This step selects the individuals or parents which have the highest fitness scores to create a new generation of potential solutions.

4. Recombination: During recombination, the algorithm combines or recombines the genetic material of the selected parents and creates a new solution.

5. Mutation: After generating a new solution through recombination, a mutation can be applied to some of the offspring randomly to introduce diversity into the population.

6. Evaluation of children: The offspring generated from crossover and mutation are evaluated for their fitness.

7. Selection of the next population: In this step, the algorithm selects the individuals from the original and newly created populations which will form the next generation.

The runtime of a genetic algorithm continues repeating steps 2-7 until either a satisfactory solution is found or a maximum number of iterations is reached.
### Context
- Topics 5&6: Creatures::Lesson 4 Introduction
- Topics 5&6: Creatures::Lecture: Introduction to Lesson 1
- Topic 2: Problem solving agents::Lecture 4: Genetic algorithms

In [35]:
question = """
c) Genetic algorithms employ a design process which is similar to the human
design process. Provide TWO arguments against this statement and TWO
arguments in support of this statement, using details of the design of genetic
algorithms.
[4 marks]
"""

questions['B_4_c'] = question
answers = display_answer(index, question)

### Question

c) Genetic algorithms employ a design process which is similar to the human
design process. Provide TWO arguments against this statement and TWO
arguments in support of this statement, using details of the design of genetic
algorithms.
[4 marks]

### Answer


Sorry, but the given context does not provide any information to answer the question.
### Generic Answer


Two arguments against this statement are:

1. Genetic algorithms operate on a different paradigm than the human design process. Whereas human design relies on intuition, creativity, and experience, genetic algorithms operate on a process of trial and error, where designs are tested and modified based on their fitness in a particular environment. This means that the design process of genetic algorithms is fundamentally different than that of humans, thus making it difficult to compare them.

2. Genetic algorithms lack the capacity for creativity and innovation, which are hallmarks of the human design process. While genetic algorithms can produce efficient and effective solutions, they are limited to variations on existing solutions and cannot produce designs that are truly novel or groundbreaking.

Two arguments in support of this statement are:

1. Genetic algorithms mimic the process of natural selection, which is fundamental to the evolution of life. By applying the principles of natural selection to design, genetic algorithms can produce solutions that are well-adapted to their environment, much in the same way that organisms evolve to survive in their environment.

2. The design process of genetic algorithms is iterative, responsive to feedback, and can rapidly produce a large number of potential solutions. This approach mirrors the problem-solving process of humans, who must identify problems, generate potential solutions, and test and refine them until a satisfactory solution is found.
### Textbook Answer


Two arguments against this statement are:

1. Genetic algorithms rely on a trial-and-error approach, whereas human design often involves a more deliberate and thoughtful planning process. Genetic algorithms use probabilistic methods to generate random solutions and then select the fittest individuals through fitness evaluation and reproduction, based on the assumption that over time, the population will converge to an optimal solution. However, this approach lacks the creativity and ingenuity characteristic of human design processes, such as problem formulation, analysis, synthesis, and evaluation. For example, genetic algorithms may converge prematurely to a suboptimal solution or get stuck in local optima, unable to explore the full search space.

2. Genetic algorithms are limited by the quality of the fitness function, which is often defined by humans based on their subjective judgment or domain expertise, and may not capture all the relevant aspects of the problem. Human design, on the other hand, can use multiple criteria and feedback loops to refine and improve the design over time. Moreover, genetic algorithms are computationally expensive and require the use of evolution operators such as crossover, mutation, and selection, which may not always be biologically plausible or efficient.

Two arguments in support of this statement are:

1. Genetic algorithms are inspired by the principles of natural selection and evolution, which have been shown to be effective in producing adaptive and robust designs in nature. The genetic algorithm mimics the natural process of reproduction and mutation, encoding the solutions as strings of genetic information, and applying genetic operators that mirror the mechanisms of variation, inheritance, and selection. This approach can generate a vast number of potential designs that can be evaluated objectively through a fitness function, without any prior assumptions or biases based on human intuition or experience.

2. Genetic algorithms can handle complex and non-linear problems that may be difficult for human designers to solve. In many cases, the search space may be too large or too convoluted for human intuition to explore, or the problem may involve multiple conflicting objectives or constraints that are hard to balance. Genetic algorithms can use parallel processing and adaptive techniques such as elitism, niching, and diversity preservation to efficiently navigate the search space and converge to a satisfactory solution. Moreover, genetic algorithms can learn from past experiences and improve their performance over time, by adjusting the parameters of the algorithm or introducing new features based on feedback.
### Context
- Topics 5&6: Creatures::Lecture: Lesson 1 summary
- Topics 5&6: Creatures::Lecture: Introduction to Lesson 1
- Topic 2: Problem solving agents::Lecture 4: Genetic algorithms

In [ ]:
message_history = [{
    'role': 'user',
    'content': """
        In this module, we have adopted an agent-based approach for considered AI systems.
        Your task is to propose an AI agent and provide its description (~1-2 pages). A
        description of an AI agent should include specification of its tasks, environment,
        background knowledge (if any), actions, performance measures.    
    """
}]

question = """
Propose an AI agent. For example, a birthday agent can monitor when birthdays of
your friends, relatives and colleagues are, advise you what birthday presents to buy
depending on preferences and availability, organise their purchase and delivery.
"""

questions['C_5_a'] = question
answers = display_answer(index, question, message_history=message_history)

# update the message history
message_history.append({ 'role': 'user', 'content': question })
message_history.append({ 'role': 'assistant', 'content': answers['textbook'] })

In [ ]:
question = """
• Aim: what is the purpose for the development of your AI agent? What tasks
will it be it able to work on?
[10 marks]
"""

questions['C_5_b'] = question
answers = display_answer(index, question, message_history=message_history)

# update the message history
message_history.append({ 'role': 'user', 'content': question })
message_history.append({ 'role': 'assistant', 'content': answers['textbook'] })

In [ ]:
question = """
• Motivation: why the proposed agent is important to develop? Who will
benefit from the development of the proposed AI agent?
[5 marks]
"""

questions['C_5_c'] = question
answers = display_answer(index, question, message_history=message_history)

# update the message history
message_history.append({ 'role': 'user', 'content': question })
message_history.append({ 'role': 'assistant', 'content': answers['textbook'] })

In [ ]:
question = """
Environment: describe the environment where your agent will be acting. Is it
deterministic, not deterministic, observable, partially observable, etc.?
Explain, why do you think so? 
[10 marks]
"""

questions['C_5_d'] = question
answers = display_answer(index, question, message_history=message_history)

# update the message history
message_history.append({ 'role': 'user', 'content': question })
message_history.append({ 'role': 'assistant', 'content': answers['textbook'] })

In [ ]:
question = """
• Functionality: briefly describe the workings of your agent. Will be it a standalone system? Can it be integrated with other AI agents or systems?
[5 marks]
"""

questions['C_5_e'] = question
answers = display_answer(index, question, message_history=message_history)

# update the message history
message_history.append({ 'role': 'user', 'content': question })
message_history.append({ 'role': 'assistant', 'content': answers['textbook'] })

In [ ]:
question = """
Data and knowledge: Will your agent use dataset(s) and knowledge? If so,
describe those data, knowledge and how they can be obtained. If not,
explain why using data and knowledge is not required.
[5 marks]
"""

questions['C_5_f'] = question
answers = display_answer(index, question, message_history=message_history)

# update the message history
message_history.append({ 'role': 'user', 'content': question })
message_history.append({ 'role': 'assistant', 'content': answers['textbook'] })

In [ ]:
question = """
• Evaluation: Explain what performance measures can be used to evaluate
your agent.
[5 marks]
"""

questions['C_5_g'] = question
answers = display_answer(index, question, message_history=message_history)

In [ ]:
question = """
In this module, we have adopted an agent-based approach for considered AI systems.
Your task is to propose an AI agent and provide its description (~1-2 pages). A
description of an AI agent should include specification of its tasks, environment,
background knowledge (if any), actions, performance measures.

Propose an extension or a modification of one of the considered agents or AI
systems. For example, you can modify the vacuum cleaner example by proposing an
intelligent vacuum cleaner that can decide what to clean, how and when. It can be
integrated with an intelligent shopping agent for buying cleaning materials.
Integration with a calendar agent can help with scheduling of cleaning, for example
before the owner is back from holidays. 

A description of your modification or extension must include the following:

• A brief description of the original agent or system: its tasks, functionality,
environment, its limitations.
[5 marks]
"""

questions['C_6_a'] = question
answers = display_answer(index, question)

In [ ]:
question = """
In this module, we have adopted an agent-based approach for considered AI systems.
Your task is to propose an AI agent and provide its description (~1-2 pages). A
description of an AI agent should include specification of its tasks, environment,
background knowledge (if any), actions, performance measures.

Propose an extension or a modification of one of the considered agents or AI
systems. For example, you can modify the vacuum cleaner example by proposing an
intelligent vacuum cleaner that can decide what to clean, how and when. It can be
integrated with an intelligent shopping agent for buying cleaning materials.
Integration with a calendar agent can help with scheduling of cleaning, for example
before the owner is back from holidays. 

A description of your modification or extension must include the following:

• Aim of the modified or extended agent: what is the purpose for the
development of your agent? What additional tasks will it be it able to work on?
[5 marks]
"""

questions['C_6_b'] = question
answers = display_answer(index, question)

In [ ]:
question = """
In this module, we have adopted an agent-based approach for considered AI systems.
Your task is to propose an AI agent and provide its description (~1-2 pages). A
description of an AI agent should include specification of its tasks, environment,
background knowledge (if any), actions, performance measures.

Propose an extension or a modification of one of the considered agents or AI
systems. For example, you can modify the vacuum cleaner example by proposing an
intelligent vacuum cleaner that can decide what to clean, how and when. It can be
integrated with an intelligent shopping agent for buying cleaning materials.
Integration with a calendar agent can help with scheduling of cleaning, for example
before the owner is back from holidays. 

A description of your modification or extension must include the following:

• Environment: describe the environment where your agent will be acting. Is it
deterministic, not deterministic, observable, partially observable, etc.? Is it
different from the original environment? Explain, why do you think so? 
[10 marks]
"""

questions['C_6_c'] = question
answers = display_answer(index, question)

In [ ]:
question = """
In this module, we have adopted an agent-based approach for considered AI systems.
Your task is to propose an AI agent and provide its description (~1-2 pages). A
description of an AI agent should include specification of its tasks, environment,
background knowledge (if any), actions, performance measures.

Propose an extension or a modification of one of the considered agents or AI
systems. For example, you can modify the vacuum cleaner example by proposing an
intelligent vacuum cleaner that can decide what to clean, how and when. It can be
integrated with an intelligent shopping agent for buying cleaning materials.
Integration with a calendar agent can help with scheduling of cleaning, for example
before the owner is back from holidays. 

A description of your modification or extension must include the following:

• Functionality: briefly describe the workings of your modified/extended agent.
What is the difference with the original agent/system?
[10 marks]
"""

questions['C_6_d'] = question
answers = display_answer(index, question)

In [ ]:
question = """
In this module, we have adopted an agent-based approach for considered AI systems.
Your task is to propose an AI agent and provide its description (~1-2 pages). A
description of an AI agent should include specification of its tasks, environment,
background knowledge (if any), actions, performance measures.

Propose an extension or a modification of one of the considered agents or AI
systems. For example, you can modify the vacuum cleaner example by proposing an
intelligent vacuum cleaner that can decide what to clean, how and when. It can be
integrated with an intelligent shopping agent for buying cleaning materials.
Integration with a calendar agent can help with scheduling of cleaning, for example
before the owner is back from holidays. 

A description of your modification or extension must include the following:

• Data and knowledge: Will your modified/ extended agent use additional
dataset(s) and knowledge? If so, describe those data, knowledge and how
they can be obtained. If not, explain why using data and knowledge is not
required.
[5 marks]
"""

questions['C_6_e'] = question
answers = display_answer(index, question)

In [ ]:
question = """
In this module, we have adopted an agent-based approach for considered AI systems.
Your task is to propose an AI agent and provide its description (~1-2 pages). A
description of an AI agent should include specification of its tasks, environment,
background knowledge (if any), actions, performance measures.

Propose an extension or a modification of one of the considered agents or AI
systems. For example, you can modify the vacuum cleaner example by proposing an
intelligent vacuum cleaner that can decide what to clean, how and when. It can be
integrated with an intelligent shopping agent for buying cleaning materials.
Integration with a calendar agent can help with scheduling of cleaning, for example
before the owner is back from holidays. 

A description of your modification or extension must include the following:

• Evaluation: Explain what performance measures can be used to evaluate your
modified/ extended agent.
[5 marks]
"""

questions['C_6_f'] = question
answers = display_answer(index, question)